# Algorithm 1
The algorithm in this file has time complexity dependent on number of hospitals h.
- It carries out BFS from every hospital to every node and records this path down for the first hospital.
- Subsequent BFS runs for each hospital compares the new path distance with the existing path recorded and only replaces the recorded path with the new path if there is an improvement in distance. 
- For part c) and d) only the distance and hospital node is recorded to save on memory

**Memory Usage**
- Running a graph with 2 million nodes takes about 4GB of memory

# Functions
Run all functions below.
## Function to construct adjacency list from file

In [ ]:
def construct_adjacency_list(graph_filename):
    graph_file = open(graph_filename,'r')
    lines = graph_file.readlines() 
    graph_file.close() 
    adjacency_list = {}
    for line in lines[4:]:
        from_node = int(line.split()[0])
        to_node = int(line.split()[1])
        if  (from_node not in adjacency_list.keys()):
            adjacency_list[from_node] = [to_node]
        elif (to_node not in adjacency_list[from_node]):
            adjacency_list[from_node].append(to_node)
        #For undirected graph
        if  (to_node not in adjacency_list.keys()):
            adjacency_list[to_node] = [from_node]
        elif (from_node not in adjacency_list[to_node]):
            adjacency_list[to_node].append(from_node)
    vertices = adjacency_list.keys()
    return adjacency_list

## Function to construct list of hospitals from hospital file

In [ ]:
#Convert hospital file to list of hospitals
def construct_hospital_list(hospital_filename):
    hospital_file = open(hospital_filename,'r')
    hospital_lines = hospital_file.readlines() 
    hospital_file.close()
    num_of_hospitals = int(hospital_lines[0].split()[1])
    hospitals_list = []
    for line in hospital_lines[1:]:
        hospitals_list.append(int(line))
    return hospitals_list

### Part a)
Computes the distance from each node in the graph to its nearest hospital. Output the distance and the shortest path for each node to a file.

In [ ]:
from tqdm import tqdm
def shortest_path_to_hospital(adjacency_list, hospital_list): 
  final_paths = {} # Final dictionary of shortest paths for each vertex
  hospital_number = 0 # To track the number of hospitals we have visited
  for hospital_node in tqdm(hospital_list):
    hospital_number+=1
    visited = { i : False for i in adjacency_list.keys() }
    visited[hospital_node] = True
    # If it is the first hospital, just add the path into the final_paths
    if (hospital_number==1):
      queue = [hospital_node] # Queue of nodes
      final_paths[hospital_node]=[hospital_node]
      while (len(queue)!= 0):
        current_node = queue.pop(0)
        if (current_node in adjacency_list.keys()): # If it has neighbours
          for neighbour_node in adjacency_list[current_node]: 
            if (not visited[neighbour_node]):
              visited[neighbour_node]=True # Mark as visited
              queue.append(neighbour_node) # Add to node to queue
              # Each time new nodes are added to the start of the path since we are traversing the graph in the opposite direction
              new_path = [neighbour_node]+ final_paths[current_node] # path = [1,2,3]
              final_paths[neighbour_node] = new_path # Add path to final_paths
    else:
      final_paths[hospital_node] = [hospital_node] # Shortest path to a hospital is itself
      path_queue = [[hospital_node]] # Queue of paths
      while (len(path_queue)!= 0):
        current_path = path_queue.pop(0) # First path in the queue
        # If the last visited node in the path has neightbours
        # Last visited node is at the start since we are traversing in the opposite direction
        if (current_path[0] in adjacency_list.keys()):  
          # Visit neighbours of the last visited node in the path
          for neighbour_node in adjacency_list[current_path[0]]:
            # If the neighbour has not been visited
            if (not visited[neighbour_node]):
              # Visit it
              visited[neighbour_node]= True
              # New path = [1,2,3]
              # Each time new nodes are added to the start of the path since we are traversing the graph in the opposite direction
              new_path = [neighbour_node] + current_path
              # New distance
              new_distance = len(new_path) - 1 #Since path includes the starting node -1
              # Append this path to the queue
              queue.append(new_path) 
              # If there hasn't been a path to this node yet
              if (neighbour_node not in final_paths.keys()):
                # Add this path
                final_paths[neighbour_node] = new_path
              # If there is an existing path to a hospital, compare the distance of the new path to the previous path
              elif (len(final_paths[neighbour_node])-1 > new_distance): # Since path includes the starting node -1
                final_paths[neighbour_node] = new_path  
  return final_paths

### Part c) and d)
Computing the distances from each node to top-k nearest hospitals for an input value of k. (k can be 2)

In [ ]:
from tqdm import tqdm
def nearest_k_hospital(adjacency_list, hospital_list, k):  #Top k number of hospitals
    # Final dictionary of nearest hospital and distance for each vertex
    final_hospital_and_distance = {} 
    hospital_number = 0 # Keep track of the hospital we are on
    for hospital_node in tqdm(hospital_list):
      hospital_number+=1
      visited = { i : False for i in adjacency_list.keys() }
      visited[hospital_node] = True
      # If it is the first hospital, just recorded in final_hospital_and_distance
      if (hospital_number==1):
        queue = [hospital_node] # Queue of nodes
        # (hospital_node, distance) This list is maintained in increasing order
        final_hospital_and_distance[hospital_node]=[(hospital_node, 0)] 
        while (len(queue)!= 0):
          current_node = queue.pop(0) # Node
          # If this node has neighbours
          if (current_node in adjacency_list.keys()):
            # Visit all neighbours of this node
            for neighbour_node in adjacency_list[current_node]:
              # If this neighbour has not been visited
              if (not visited[neighbour_node]):
                # Visit it
                visited[neighbour_node]=True
                #Add to queue
                queue.append(neighbour_node)
                new_distance = final_hospital_and_distance[current_node][0][1] + 1
                #For the first hospital, we can directly record it in final hospital
                final_hospital_and_distance[neighbour_node] = [(hospital_node, new_distance)] # (hospital_node , distance)
      else: #For all subsequent hospitals
        if (hospital_node in final_hospital_and_distance.keys()):
          # Always first in the list since shortest path to hospital is itself
          final_hospital_and_distance[hospital_node].insert(0, (hospital_node, 0)) 
          # Trim the list to fit k
          final_hospital_and_distance[hospital_node] = final_hospital_and_distance[hospital_node][:k] 
        else: 
          final_hospital_and_distance[hospital_node] = (hospital_node, 0)
        
        # Queue of (current_node, distance) 
        distance_queue = [(hospital_node, 0)] 
        while (len(distance_queue)!= 0):
          current_node_dist = distance_queue.pop(0) # Path
          # If the current node has neightbours
          if (current_node_dist[0] in adjacency_list.keys()):
            # Visit neighbours of the current node
            for neighbour_node in adjacency_list[current_node_dist[0]]:
              # If the neighbour has not been visited
              if (not visited[neighbour_node]):
                # Visit it
                visited[neighbour_node]= True
                # New distance = previous distance + 1
                new_distance = current_node_dist[1] + 1
                # Append this path to the queue
                distance_queue.append((neighbour_node, new_distance)) #(current_node, distance)
                # If there hasn't been a hospital to this node yet
                if (neighbour_node not in final_hospital_and_distance.keys()):
                  # Add this path
                  final_hospital_and_distance[neighbour_node] = [(hospital_node, new_distance)]
                # If there is an existing path to a hospital
                # Compare the distance of the new path to all previous entries from the right to the left 
                # to find position to place the new entry (Like insertion sort)
                # Entries are in increasing order of distance
                else:
                  # Compare distance from right to left
                  for i in range(len(final_hospital_and_distance[neighbour_node])-1, -1, -1):
                    if (final_hospital_and_distance[neighbour_node][i][1] <= new_distance):
                      final_hospital_and_distance[neighbour_node].insert(i+1, (hospital_node, new_distance))
                      break
                    # If it is the shortest path
                    if (i==0 and final_hospital_and_distance[neighbour_node][i][1] > new_distance): 
                      #Place it at the start
                      final_hospital_and_distance[neighbour_node].insert(0,(hospital_node, new_distance))
                  #Trim the list to fit k
                  final_hospital_and_distance[neighbour_node] = final_hospital_and_distance[neighbour_node][:k] 
    return final_hospital_and_distance

## Path Finding UI


### Loading files

**Graph File**

Make sure the format of the graph file matches this example:
```
Line0: # Directed graph (each unordered pair of nodes is saved once): roadNet-CA.txt
Line1: # California road network
Line2: # Nodes: 1965206 Edges: 5533214
Line3: # FromNodeId ToNodeId
Line4: 0 1
Line5: 0 2
```





In [ ]:
# Loading graph file
import time
graph_filename = input("Graph filename: ")
adjacency_list_construction_start_time = time.time()
adjacency_list = construct_adjacency_list(graph_filename)
adjacency_list_construction_time = time.time() - adjacency_list_construction_start_time
print("Adjacency List Constructed in", adjacency_list_construction_time, "seconds")

**Hospital file**

Make sure the format of the hospital file matches this example:
```
Line0: # 3
Line1: 6
Line2: 11
Line3: 4
```

In [ ]:
# Loading hospital file
import time
hospital_filename = input("Hospital filename: ")
hospital_list_construction_start_time = time.time()
hospital_list = construct_hospital_list(hospital_filename)
hospital_list_construction_time = time.time() - hospital_list_construction_start_time
print("Hospital List Constructed in", hospital_list_construction_time, "seconds")

### Part a)
Computes the distance from each node in the graph to its nearest hospital. Output the distance and the shortest path for each node to a file.

**Make sure you have loaded the files above**

In [ ]:
import time
from tqdm import tqdm
output_filename = input("Name of output file: ")
shortest_path_start_time = time.time()
shortest_path_output = shortest_path_to_hospital(adjacency_list, hospital_list)
f = open(output_filename, "w")
all_nodes = list(adjacency_list.keys())
for i in tqdm(range(len(all_nodes))):
  node = all_nodes.pop(0)
  if (node in shortest_path_output.keys()):
    path = shortest_path_output[node]
    distance = len(path) - 1 # Exclude starting node
    # Output to file
    print("Node: ", node, file=f)
    print("Path:", path , file=f)
    print("Distance: ", distance, file=f)
    print("---------------------------------------------------", file=f)
    # Output to console
    # print("Node: ", node)
    # print("Path:", path)
    # print("Distance: ", distance)
    # print("---------------------------------------------------")
    shortest_path_output.pop(node)
  else:
    print("No paths to hospital from node", node, file=f)
    # print("No paths to hospital from node", node)
f.close()
shortest_path_time = time.time() - shortest_path_start_time
print("Shortest Path Ouputed in", shortest_path_time, "seconds")

### Part c) and d)
Computing the distances from each node to top-k nearest hospitals for an input value of k. (k can be 2)

**Make sure you have loaded the files above**

In [ ]:
import time
from tqdm import tqdm
output_filename = input("Name of output file: ")
k = int(input("Value of k: "))
nearest_hospital_start_time = time.time()
nearest_hospital_output = nearest_k_hospital(adjacency_list, hospital_list, k)
f = open(output_filename, "w")
all_nodes = list(adjacency_list.keys())
for i in tqdm(range(len(all_nodes))):
  node = all_nodes.pop(0)
  if (node in nearest_hospital_output.keys()):
      hospital_dist_list = nearest_hospital_output[node] # [(Hospital, Distance)]
      # Output to file
      print("Node: ", node, file=f)
      # print("Node: ", node)
      print("Nearest Hospitals", k, ": ", file=f)
      # print("Nearest Hospitals", k, ": ")
      for hospital_node, distance in hospital_dist_list:
        print("Hospital", hospital_node, "\tDistance:", distance, file=f) 
        # print("Hospital", hospital_node, "\tDistance:", distance) 
      print("---------------------------------------------------", file=f)
      # print("---------------------------------------------------")
      nearest_hospital_output.pop(node)
  else:
    print("No paths to hospital from node", node, file=f)
    # print("No paths to hospital from node", node)
f.close()
nearest_hospital_time = time.time() - nearest_hospital_start_time
print("Nearest Hospitals Ouputed In", nearest_hospital_time, "seconds")